#  Exercise 01
The goal is to find the best set of hyper-parameters which maximize the
performance on a training set.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# This line is currently required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from scipy.stats import expon, uniform
from scipy.stats import randint

df = pd.read_csv("https://www.openml.org/data/get_csv/1595261/adult-census.csv")
# Or use the local copy:
# df = pd.read_csv('../datasets/adult-census.csv')

target_name = "class"
target = df[target_name].to_numpy()
data = df.drop(columns=target_name)

df_train, df_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

You should:
- create a preprocessor using an `OrdinalEncoder`
- use a `HistGradientBoostingClassifier` to make predictions
- use a `RandomizedSearchCV` to find the best set of hyper-parameters by
  tuning the following parameters: `learning_rate`, `l2_regularization`,
  `max_leaf_nodes`, and `min_samples_leaf`.

In [2]:
ordinal_encoding_columns = ['workclass', 'education', 'marital-status',
                            'occupation', 'relationship', 'race',
                            'native-country', 'sex']

categories = [data[column].unique()
              for column in data[ordinal_encoding_columns]]

preprocessor = ColumnTransformer(
    [('ordinal-encoder', OrdinalEncoder(categories=categories),
      ordinal_encoding_columns)],
    remainder='passthrough', sparse_threshold=0
)

model = Pipeline(
    [('preprocessor', preprocessor),
     ('gbrt', HistGradientBoostingClassifier(max_iter=50))]
)
param_distributions = {
    'gbrt__learning_rate': expon(loc=0.001, scale=0.5),
    'gbrt__l2_regularization': uniform(loc=0, scale=0.5),
    'gbrt__max_leaf_nodes': randint(5, 30),
    'gbrt__min_samples_leaf': randint(5, 30)
}
model_grid_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10, n_jobs=4
)
model_grid_search.fit(df_train, target_train)
print(
    f"The accuracy score using a {model_grid_search.__class__.__name__} is "
    f"{model_grid_search.score(df_test, target_test):.2f}"
)
print(f"The best set of parameters is: {model_grid_search.best_params_}")

The accuracy score using a RandomizedSearchCV is 0.88
The best set of parameters is: {'gbrt__l2_regularization': 0.2663485156658373, 'gbrt__learning_rate': 0.17764488545026633, 'gbrt__max_leaf_nodes': 25, 'gbrt__min_samples_leaf': 9}


In [3]:
df_results = pd.DataFrame(model_grid_search.cv_results_)
columns = (['mean_test_score', 'std_test_score'] +
           [col for col in df_results.columns if 'param_' in col])
df_results.sort_values(by='mean_test_score', ascending=False)[
    columns
]

,mean_test_score,std_test_score,param_gbrt__l2_regularization,param_gbrt__learning_rate,param_gbrt__max_leaf_nodes,param_gbrt__min_samples_leaf
7,0.870956,0.002648,0.266349,0.177645,25,9
4,0.870110,0.001766,0.306641,0.145503,22,18
3,0.869509,0.002394,0.194557,0.308305,24,28
5,0.868772,0.001833,0.390562,0.394437,19,12
2,0.866698,0.002433,0.0607027,0.811675,5,9
6,0.863695,0.003321,0.337931,0.581752,25,29
9,0.861647,0.003757,0.271786,0.582775,29,11
0,0.855423,0.004731,0.35025,1.31498,12,14
1,0.814829,0.002161,0.151998,0.0154335,10,25
8,0.737817,0.014355,0.436514,1.83281,11,9
